# 本notebook是为了记录第三届AI+创新创业大赛：半监督学习目标定位竞赛第5名方案的

比赛链接：[https://aistudio.baidu.com/aistudio/competition/detail/78](https://aistudio.baidu.com/aistudio/competition/detail/78)

## 一.比赛整体把握

比赛的赛题是半监督学习目标定位，是为了把图片中物品分割出来。之前做过眼球，肾脏，遥感图相关的图像分割，这里就沿用之前的方法使用PaddleSeg套件。

## 二.方案思路介绍

由于参赛时间较短(比赛前一周才报名),所以在方案上并没有创新的地方，没有使用原创的网络也没有原创的backbone（看了第一名的方案感觉我好菜啊）

### 1.模型

使用PaddleSeg已有的模型进行比赛，在此基础上，不断优化模型和骨干网络：

成绩由低到高，分别尝试了DeepLabV3-Resnet50,DeepLabV3-Resnet101，之后DeepLabV3P-Resnet50，效果提升了一点；然后尝试了OCRNet-Hrnetw48，计算消耗资源多了一倍，但是效果有提升，最终选定了这个模型。

OCRnet介绍：[论文地址](https://arxiv.org/pdf/1909.11065.pdf)

OCRNet的含义是使用了OCR这个方法的网路，英文全称Object-Contextual Representations，这个词能看出来，这是对分割物体的重视。

摘要中对OCR的解释：也就是说利用了每一个像素所属的物体类别来进行分割。
```
In this paper, we study the context aggregation problem in
semantic segmentation. Motivated by that the label of a pixel is the
category of the object that the pixel belongs to, we present a simple
yet effective approach, object-contextual representations, characterizing a pixel by
exploiting the representation of the corresponding object class.
```
具体的方法可以概括为：
```
First, we learn object regions under the supervision of the ground-truth segmentation.
Second, we compute the object region representation by aggregating the representations of the pixels lying in the object region. 
Last, we compute the relation between each pixel and each object region, and augment the representation of each pixel
with the object-contextual representation which is a weighted aggregation of all the object region representatio
```
也就是首先形成软物体区域（Soft Object Regions），然后计算物体区域表示（Object Region Representations），最后得到物体上下文特征表示和上下文信息增强的特征表示（Augmented Representation）。

OCR模块的结构：

![](https://ai-studio-static-online.cdn.bcebos.com/a09f1bb266b4434a904256fcc87020ccfa7a02eddb9a4f51b6c74604d0d33b09)


### 2.调参

通过调整学习率、训练轮数、多尺度等超参数来提升成绩，学习率和训练轮数是靠着观察训练日志中miou来调整的，而多尺度的调整效果不好，在训练中增加了后，单模型分数下降了1分

轮数方面：训练iters由10000到50000都尝试过，太多还是容易导致过拟合的。

学习率方面，PaddleSeg主要通过lr_scheduler来控制学习率

```
lr_scheduler:
  type: PolynomialDecay
  learning_rate: 0.0013
  end_lr: 0
  power: 0.9
```
初始学习率在0.002和0.0005之间调整

### 3.预处理

观察数据集后，采用了更多的数据预处理方法，这里使用了PaddleSeg的transform模块，在config文件中的体现是：

```
  transforms:
    - type: RandomHorizontalFlip
    - type: RandomVerticalFlip
    - type: ResizeStepScaling
      min_scale_factor: 0.45
      max_scale_factor: 2.6
      scale_step_size: 0.2
    - type: RandomRotation
      max_rotation: 17
    - type: RandomDistort
      brightness_range: 0.2
      contrast_range: 0.2
      saturation_range: 0.3
    - type: Resize
      target_size: [256, 256]
    - type: Normalize
```

这些参数具体的解释可以参考PaddleSeg\paddleseg\transform中的functional.py和transforms.py文件

传送门：在终端中打开：

```
vim /home/aistudio/PaddleSeg/paddleseg/transforms/functional.py
或者
vim /home/aistudio/PaddleSeg/paddleseg/transforms/transforms.py
```
通过添加ResizeStepScaling和RandomRoatation，单模型的分数可以提高0.5

Resize多尺度的效果不好，建议不要尝试

### 4.在损失函数上

在baseline和paddleseg的config文件中，常给出的损失函数是单一的损失函数，如CrossEntropyLoss

为了解决类别不均衡现象而提出的lovasz loss，Lovasz loss基于子模损失(submodular losses)的凸Lovasz扩展，对神经网络的mean IoU损失进行优化。

实际使用的时候常常与cross entropy loss或bce loss加权使用

下面是某一次训练中实际使用的配比：

```
loss:
  types:
    - type: MixedLoss
      losses:
        - type: CrossEntropyLoss
        - type: LovaszSoftmaxLoss
      coef: [0.5, 0.5]
    - type: MixedLoss
      losses:
        - type: CrossEntropyLoss
        - type: DiceLoss
      coef: [0.5, 0.5]
  coef: [1, 0.8]
```

## 三.训练过程

训练主要使用了Aistudio的notebook后台任务、单卡P40脚本任务、四卡V100脚本任务

最后一次提交的结果使用了7个模型,所以想要在notebook任务中再现训练过程，**可能需要至少49小时**
 
这里首先提供训练的日志:

训练日志截图

![](https://ai-studio-static-online.cdn.bcebos.com/27edfc578d4b40caaba11c06b08d75692a380711abd34a5087194746ef973243)


训练完成的截图

![](https://ai-studio-static-online.cdn.bcebos.com/1893cba4c7ea45e88c6db5b847574b0726e6d3a9711946ee89b02cb7ffd97a35)


在/home/aistudio/record文件夹中提供了所有模型的训练日志

可以使用终端分别查看：

```
vim record/81918/81918.log
vim record/82921/82921.log
vim record/83226/83226.log
vim record/83374/83374.log
vim record/83606/83606.log
vim record/testB01/testB1_82063.log
vim record/testB02/testB2_82669.log
```

由于训练日志中有很多是安装相关库和解压文件的，这里建议使用vim的跳行功能：数字加上下方向键，大约先下降十万两千行就能看到训练过程了

这里是我的失误，由于压缩指令没有家-oq导致产生了很多无用信息。

训练所用的config文件全部在/home/aistudio/config中

总共需要训练七个模型，下面提供了训练命令：

In [ ]:
#首先解压训练集
!unzip -qo data/data95249/train_50k_mask.zip -d data/
!unzip -oq data/data95249/train_image.zip -d data/

In [ ]:
!python config/81918/run.py

In [ ]:
!python config/82921/run.py

In [ ]:
!python config/83226/run.py

In [ ]:
!python config/83606/run.py

In [ ]:
!python config/83374/run.py

In [ ]:
!python config/testB01/run.py

In [ ]:
!python config/testB02/run.py

注意：由于之前训练模型采用的是单卡P40或者四卡V100，所以batch_size并不一样，可能会造成结果不同

所有脚本都已经固定了随机种子

## 四.预测与集成学习

### 1.预测
对于得到的模型，使用多尺度和水平垂直翻转进行预测

其中多尺度一般采用1,1.5,2;但其中也混杂了一些不规则的数字，均是作者中二病犯了随手打的，比如1.3,1.44,2.1等等。

多尺度的预测，在单模型上曾经带来约0.2的提升

![](https://ai-studio-static-online.cdn.bcebos.com/fad392e5744e49328dcd04d1567cbd55778f4794f65c40abac1e97296c6d88d5)


这里首先解压测试集和所有模型

In [ ]:
!mkdir data/models && unzip -qo data/data103435/models.zip -d data/models
!unzip -oq data/data103435/dataB.zip -d data
!mkdir data/resultfinal && unzip -qo data/data103435/result_0.78+.zip -d data/resultfinal

之后部分预测的部分代码如下：

In [ ]:
!python PaddleSeg/predict.py --config ocrnet.yml --model_path /home/aistudio/data/models/83226/model.pdparams --image_path data/test_image --save_dir check/080401 --aug_pred --flip_horizontal --flip_vertical  --scales=1.25

In [ ]:
!python PaddleSeg/predict.py --config ocrnet.yml --model_path /home/aistudio/data/models/83226/model.pdparams --image_path data/test_image --save_dir check/080402 --aug_pred --flip_horizontal --flip_vertical  --scales=1.75

预测使用的模型和多尺度的次数很多，notebook中不再全部写出来。

所有模型的预测脚本在/home/aistudio/infer.sh中

可以直接使用如下命令完成预测

```
bash infer.sh
```
但不建议这样，因为耗时太久，可能需要20h以上才能全部预测完

可以把infer.sh文件打开,命令复制到三个终端和notebook中，四个同时进行.大约耗时10小时


为什么预测了这么多次？

这说来就是我自己的愚蠢操作了，因为偷懒的缘故，每次都是使用前一天的最佳结果和最新的模型结果进行融合,这就导致在总结清算的时候，发现融合了太多的模型，投票的时候早期的模型占比甚至都只有1%，但是为了复现结果，所以就都带上了。

### 4.融合

融合的思路就是让不同的模型预测同一个点的所属类别，得到最终结果。

这里借鉴了手机相机上多帧合成的技术：为了使得图像更清晰，一次拍出多张照片，增加曝光时间，把多张照片合成一张照片。

对于得到的单模型结果，使用多次简单的融合，最终得到一个较好的结果。

如果在notebook中完成了预测，则可以运行

```
python check/ensemble.py
```


或者直接使用已经上传号的结果文件，融合时直接采用下面的脚本就可以完成



In [ ]:
!python ensemble.py

first ensemble: around 0.778
_____________________________
100%|████████████████████████████████████| 10989/10989 [01:16<00:00, 142.88it/s]
third ensemble: around 0.782
_____________________________
100%|█████████████████████████████████████| 10989/10989 [02:16<00:00, 80.24it/s]
forth ensemble: around 0.783
_____________________________
100%|████████████████████████████████████| 10989/10989 [01:42<00:00, 107.69it/s]
fifth ensemble: around 0.783
_____________________________
100%|█████████████████████████████████████| 10989/10989 [02:36<00:00, 70.26it/s]
final ensemble: around 0.7844
_____________________________
100%|█████████████████████████████████████| 10989/10989 [05:31<00:00, 33.12it/s]
finish the job


## 五.改进方向与思考

### 1.改进方向

#### 模型方面：

通过查看Paddleclas的代码仓库，才知道原来PaddleSeg库中很多模型找不到的backbone的预训练文件都可以找到

例如hrnet可选的配置有这些：
```
__all__ = [
    "HRNet_W18_Small_V1", "HRNet_W18_Small_V2", "HRNet_W18", "HRNet_W30",
    "HRNet_W32", "HRNet_W40", "HRNet_W44", "HRNet_W48", "HRNet_W60", "HRNet_W64"
]
```
PaddleSeg中只找到了w18和w48，其他的都需要到PaddleCV或者PaddleClas中去找。显然，换用更大的模型是一种提升的方式。


#### 超参数调节

PaddleSeg并没有把所有的PaddlePaddle预处理、学习率调节方式、优化器都加进去。

例如可以自行在PaddleSeg/paddleseg/transforms/transforms.py文件中添加预处理方式

#### 模型融合

应当选择，由于作者参赛时间较晚同时忙于家事，所以提交的时候没有仔细优化融合的思路。实际比赛中，应该融合更强的，同时预测结果差异更大的模型。


### 2.鬼影后处理（与比赛无关）

单模型出来的图片都是非常清楚锐利的，毕竟Peak Threshold都是240以上的

但是通过简单的融合之后，有些图形就出了鬼影

这应该是正常的，因为结果是简单投票出来的，一个模型认为A点是255，另一个模型认为这里是0.最后投票出来就是127，导致图像出现灰蒙蒙的地方

这种情况也常见于夜间手机拍照，在增加了多帧合成以后，图片整体分辨率上升，亮度增加，但是可能出现莫名其妙的色彩。

对于单张图片demp.jpg  
![](https://ai-studio-static-online.cdn.bcebos.com/ecbbed8bcbe94f67aa7d90c59dd3a16847411a5f192743d08ccc7fad29669df1)


使用
```
img1 = Image.open('demo.jpg')
img1 = np.array(img1)
print((len(img1)))

img6 = img6 = np.zeros_like(img1)
for i in range(img1.shape[0]):
    for j in range(img1.shape[1]):
        img1[i,j] = 255 if img1[i,j]>140 else 0
        img6[i,j] = np.asarray(img1[i,j])

img6 = Image.fromarray(img6)
img6.save('result.png')
```
即可得到锐化后的结果 


![](https://ai-studio-static-online.cdn.bcebos.com/4dcbce99d3e941c58992b4ecfba3fa74a1fa00741d034bb9ac3d0f5ebdc50127)

但是比赛的评价指标只是MIOU，并没有使用视觉观感的评价指标，所以这一策略没有尝试提交



## 六.参考文献

参考的项目:

[第三届中国AI+创新创业大赛：半监督学习目标定位竞赛baseline](https://aistudio.baidu.com/aistudio/projectdetail/2145019?channelType=0&channel=0)

非常感谢羽小翟大佬。使用他的baseline，直接就能达到20名左右的位置。

在这个baseline中，魔改了paddleseg核心模块中训练文件。

参考文献：

OCRnet：[论文地址](https://arxiv.org/pdf/1909.11065.pdf)
